<a href="https://colab.research.google.com/github/YooNayoung/ESAA/blob/main/0523session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Amazon Reviews를 이용한 추천시스템 실습 - surprise패키지 사용** [36~]

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
electronics_data = pd.read_csv("/content/drive/MyDrive/ESAA(OB)/ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])

In [ ]:
electronics_data.head()

In [5]:
eletronics_data = electronics_data.iloc[:1048576,0:]

In [6]:
eletronics_data.drop(['timestamp'],axis = 1, inplace = True)

In [ ]:
eletronics_data.head()

In [7]:
new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

In [11]:
new_df.head()

,userId,productId,Rating,timestamp
183,A1BKC6B7YHYTVV,0972683275,4.0,1405382400
184,AWVFSIB42LHKJ,0972683275,4.0,1405209600
185,A36MQBNADRH8YY,0972683275,5.0,1405641600
186,A3SRXMPLAEZ6DP,0972683275,4.0,1405987200
187,A20XXTXWF2TCPY,0972683275,5.0,1405123200


--------------------------------------------------------------------------------

## **Collaborative Filtering 이용**

In [ ]:
!pip3 install surprise

In [8]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

## **1) Data 가져오기**

In [9]:
new_df.drop(['timestamp'],axis = 1, inplace = True)

In [10]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(new_df,reader)

## **2) Trainset, Testset으로 나누기**

In [11]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

## **3) KNN을 이용 trainset에 파라미터 맞추기**

In [ ]:
algo = KNNWithMeans(k=5, sim_options = {'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...


## **4) Test accuracy 측정**

In [ ]:
test_pred = algo.test(testset)

In [ ]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

## **Model-based collaborative filtering system**

In [13]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

productId,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,1400532736,1400599997,1400698987,...,B00000JFMK,B00000JHWX,B00000JI4F,B00000JII6,B00000JMUG,B00000JPPI,B00000JSGF,B00000JYLO,B00000JYWQ,B00000K135
userId,,,,,,,,,,,,,,,,,,,,,
A01852072Z7B68UHLI5UG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0266076X6KPZ6CCHGVS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0293130VTX2ZXA70JQS,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A030530627MK66BD8V4LN,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0571176384K8RBNKGF8O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
ratings_matrix.shape

(9832, 76)

In [15]:
X = ratings_matrix.T
X.head()

userId,A01852072Z7B68UHLI5UG,A0266076X6KPZ6CCHGVS,A0293130VTX2ZXA70JQS,A030530627MK66BD8V4LN,A0571176384K8RBNKGF8O,A0590501PZ7HOWJKBGQ4,A0641581307AKT5MAOU0Q,A076219533YHEV2LJO988,A0821988FXKFYX53V4QG,A099626739FNCRNHIKBCG,...,AZWOPBY75SGAM,AZX0ZDVAFMN78,AZX5LAN9JEAFF,AZX7I110AF0W2,AZXKUK895VGSM,AZXP46IB63PU8,AZYTSU42BZ7TP,AZZGJ2KMWB7R,AZZMV5VT9W7Y8,AZZST8OYL5P4Q
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,0,0,5,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501520,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1400501776,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1400532620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X.shape

(76, 9832)

In [17]:
X.index

Index(['0972683275', '1400501466', '1400501520', '1400501776', '1400532620',
       '1400532655', '140053271X', '1400532736', '1400599997', '1400698987',
       '3744295508', '6301977173', '7214047977', '8862935293', '9573212919',
       '9575871979', '9625993428', '9888002198', '9966694544', '9983891212',
       '9984984354', '9985511476', 'B000001OM4', 'B000001OM5', 'B000001OMI',
       'B000001ON6', 'B00000DM9W', 'B00000IGBF', 'B00000J05A', 'B00000J061',
       'B00000J08Q', 'B00000J0D2', 'B00000J0D5', 'B00000J0D8', 'B00000J1EJ',
       'B00000J1EP', 'B00000J1EQ', 'B00000J1F3', 'B00000J1GA', 'B00000J1QK',
       'B00000J1QR', 'B00000J1SC', 'B00000J1TX', 'B00000J1U8', 'B00000J1UQ',
       'B00000J1V3', 'B00000J1V5', 'B00000J3NF', 'B00000J3Q7', 'B00000J3UJ',
       'B00000J434', 'B00000J4EY', 'B00000J4FS', 'B00000J4GE', 'B00000J6WY',
       'B00000JBAT', 'B00000JBHP', 'B00000JBPB', 'B00000JCT8', 'B00000JCTO',
       'B00000JD34', 'B00000JDF5', 'B00000JDF6', 'B00000JDHV', 'B00000JFE3',

In [18]:
X1 = X

In [19]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(76, 10)

In [ ]:
decomposed_matrix

In [21]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(76, 76)

In [22]:
X.index[75]

'B00000K135'

In [23]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i) #index구하기
product_ID

75

In [24]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(76,)

In [25]:
correlation_product_ID

array([-0.08720386, -0.35048721, -0.26480115,  0.00500702, -0.05567627,
        0.00361999, -0.14615248, -0.33600598, -0.15345176, -0.41626816,
        0.18563619, -0.69412057, -0.65346842,  0.96444383,  0.4142683 ,
        0.87708423,  0.28015769,  0.13517441, -0.07507763,  0.95827887,
        0.36365793, -0.78602688, -0.93178906, -0.49849212, -0.0242969 ,
        0.9026086 , -0.08018657,  0.32285954,  0.4428907 , -0.01758544,
        0.96931643,  0.91621176, -0.90862766, -0.47699986, -0.8898215 ,
        0.85070277,  0.44617449,  0.64231867, -0.10477309, -0.84363201,
        0.03794457, -0.27075994, -0.16790969,  0.67576472,  0.78729939,
       -0.14751971, -0.09885805,  0.25402716,  0.05787027, -0.91680389,
        0.39068915, -0.22419902,  0.40490343,  0.46526404,  0.50977592,
        0.83386873,  0.94042859,  0.01328247,  0.45380158,  0.53381893,
       -0.18705129, -0.19994621, -0.24029134,  0.08271723, -0.97839326,
       -0.73616907,  0.14519022, -0.72525203, -0.009223  , -0.72

In [26]:
Recommend = list(X.index[correlation_product_ID > 0.65]) #피어슨 상관계수가 0.65보다 큰 상품들의 index의 list return

Recommend.remove(i) # 이미 구입한 상품은 뺀다

Recommend[0:24] #상위 24개 추천

['8862935293',
 '9575871979',
 '9983891212',
 'B000001ON6',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J1EP',
 'B00000J1U8',
 'B00000J1UQ',
 'B00000JBAT',
 'B00000JBHP',
 'B00000JPPI']

In [27]:
Recommend

['8862935293',
 '9575871979',
 '9983891212',
 'B000001ON6',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J1EP',
 'B00000J1U8',
 'B00000J1UQ',
 'B00000JBAT',
 'B00000JBHP',
 'B00000JPPI']